## Handling missing values

1. impute them with some values
2. delete those samples

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/car-sales-extended-missing-data.csv');df

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


In [3]:
df.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

### 1. Fill NaN with pandas

In [11]:
#fill features
df.Make.fillna('missing',inplace=True)

df.Colour.fillna('missing',inplace=True)

df['Odometer (KM)'].fillna(df['Odometer (KM)'].mean(), inplace=True)

df.Doors.fillna(4, inplace=True)

In [ ]:
#remove target missing rows

In [13]:
df.dropna(inplace=True)

In [14]:
df.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

In [15]:
x = df.drop('Price',axis=1)
y = df.Price

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_feature = ['Make','Colour','Doors']

one_hot = OneHotEncoder()

transformer = ColumnTransformer([('one_hot', one_hot, cat_feature)],remainder='passthrough')

new = transformer.fit_transform(df)
pd.DataFrame(new)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,35431.0,15323.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0,19943.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,84714.0,28343.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,154365.0,13434.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0,14043.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35820.0,32042.0
946,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,155144.0,5716.0
947,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,66604.0,31570.0
948,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,215883.0,4001.0


## 2. Fill values using Scikit

In [20]:
df = pd.read_csv('data/car-sales-extended-missing-data.csv')

In [21]:
#drop the rows with no values
df.dropna(subset = ['Price'], inplace=True)
df.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [22]:
#split x,y
x = df.drop('Price',axis=1)
y = df.Price

#split into test train
from sklearn.model_selection import train_test_split
np.random.seed(42)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [23]:
x.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
dtype: int64

Let's fill the missing values. We'll fill the training and test values separately to ensure training data stays with the training data and test data stays with the test data.

Note: We use fit_transform() on the training data and transform() on the testing data. In essence, we learn the patterns in the training set and transform it via imputation (fit, then transform). Then we take those same patterns and fill the test set (transform only).

In [35]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Fill categorical values with 'missing' & numerical values with mean
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
door_imputer = SimpleImputer(strategy='constant',fill_value=4)
num_imputer =SimpleImputer(strategy='mean')

# Define columns
cat_feature = ['Make','Colour']
door_feature = ['Doors']
num_feature = ['Odometer (KM)']

# Create an imputer (something that fills missing data)
imputer = ColumnTransformer([('cat_imputer',cat_imputer,cat_feature),
           ('door_imputer',door_imputer,door_feature),
           ('num_imputer',num_imputer,num_feature)])

# Fill train and test values separately
filled_x_train = imputer.fit_transform(x_train) # fit_transform imputes the missing values from the training set and fills them simultaneously
filled_x_test  = imputer.transform(x_test) # tranform takes the imputing missing values from the training set and fills the test set with them

# Check filled X_train
filled_x_train

array([['Honda', 'White', 4.0, 71934.0],
       ['Toyota', 'Red', 4.0, 162665.0],
       ['Honda', 'White', 4.0, 42844.0],
       ...,
       ['Toyota', 'White', 4.0, 196225.0],
       ['Honda', 'Blue', 4.0, 133117.0],
       ['Honda', 'missing', 4.0, 150582.0]], dtype=object)

In [38]:
# Get our transformed data array's back into DataFrame's
df_filled_xtrain = pd.DataFrame(filled_x_train, columns=["Make", "Colour", "Doors", "Odometer (KM)"])
df_filled_xtest = pd.DataFrame(filled_x_test, columns=["Make", "Colour", "Doors", "Odometer (KM)"])

In [40]:
df_filled_xtrain

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4.0,71934.0
1,Toyota,Red,4.0,162665.0
2,Honda,White,4.0,42844.0
3,Honda,White,4.0,195829.0
4,Honda,Blue,4.0,219217.0
...,...,...,...,...
755,Toyota,missing,4.0,218803.0
756,BMW,Blue,5.0,245427.0
757,Toyota,White,4.0,196225.0
758,Honda,Blue,4.0,133117.0


In [41]:
df_filled_xtest

,Make,Colour,Doors,Odometer (KM)
0,Toyota,Blue,4.0,99761.0
1,Toyota,Black,4.0,17975.0
2,Honda,Blue,4.0,197664.0
3,Nissan,Green,4.0,235589.0
4,Honda,Black,4.0,231659.0
...,...,...,...,...
185,Honda,Green,4.0,165101.0
186,Nissan,Green,3.0,153554.0
187,Toyota,White,4.0,108569.0
188,BMW,Black,3.0,201190.0


In [43]:
df_filled_xtest.isna().sum(), df_filled_xtrain.isna().sum()

(Make             0
 Colour           0
 Doors            0
 Odometer (KM)    0
 dtype: int64,
 Make             0
 Colour           0
 Doors            0
 Odometer (KM)    0
 dtype: int64)

In [44]:
#one hot encoding
from sklearn.preprocessing import OneHotEncoder

cat_features = ["Make", "Colour", "Doors"]
one_hot = OneHotEncoder()

transformer = ColumnTransformer([('one_hot',one_hot,cat_features)], remainder='passthrough')

# Fill train and test values separately
transformed_x_train = transformer.fit_transform(df_filled_xtrain)
transformed_x_test = transformer.transform(df_filled_xtest)

In [46]:
transformed_x_test.toarray()

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 9.97610e+04],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.79750e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.97664e+05],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.08569e+05],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 2.01190e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.11630e+04]])

## Fit a model
Now we've filled and transformed our data, ensuring the training and test sets have been kept separate. Let's fit a model to the training set and evaluate it on the test set.

In [52]:
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor().fit(transformed_x_train, y_train)
clf.score(transformed_x_test, y_test)

0.21229043336119102